In [25]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use
import torch
from kobart import get_kobart_tokenizer
from transformers.models.bart import BartForConditionalGeneration
import re

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def load_model(path):
    model = BartForConditionalGeneration.from_pretrained(path)
    return model

In [4]:
model = load_model('/etc/jupyterhub/pythonex/Paper_2023/KoBART/model')
model = model.to(DEVICE)
tokenizer = get_kobart_tokenizer()

using cached model. /etc/jupyterhub/pythonex/Paper_2023/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip


In [141]:
reg_del = re.compile('\([a-zA-Z]{1}')
reg_sp = re.compile('\(')
reg_ep = re.compile('\)')
reg_headnum = re.compile('[\d+]\.|\(+\d+\)+')
def after_proc(text):    
    print(text)
    temp = text.split();        
    if (len(temp[0]) == 1 and temp[0] != 'I' and temp[0] != 'A' or temp[0].isdigit() or reg_headnum.search(temp[0]) != None ):
        print(1)
        text = ' '.join(temp[1:])
        text = after_proc(text)            
        temp = text.split()
    
    if (reg_del.search(temp[0]) != None and len(reg_sp.findall(text)) != len(reg_ep.findall(text)) ):
        print(len(reg_sp.findall(text)), len(reg_ep.findall(text)))
        #text = f'{temp[0]} {temp[1].replace("(", "")} {" ".join(temp[2:])}'
        text = f'{temp[0].replace("(", "")} {" ".join(temp[1:])}'
        temp = text.split()
        print(text)        
        
    if (temp[1].find(temp[0]) == 0) :        
        print(2)
        text = ' '.join(temp[1:])  
        temp = text.split()
    
    if (' '.join(temp[2:4]).find(' '.join(temp[0:2])) ==0) :        
        print(3)
        text = ' '.join(temp[2:])  
    
    return text

reg_hang = re.compile('^[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚]|^\([\d]{1,2}\)')
reg_ho= re.compile('^\d+\.|^\d+\.')
reg_mog = re.compile('^[가-호]{1}\.|^\([a-zA-Z]\)')
headnum_dict = {}
def create_headnum_dic():
    headnum_dict['①'] = '(1)'
    headnum_dict['②'] = '(2)'
    headnum_dict['③'] = '(3)'
    headnum_dict['④'] = '(4)'
    headnum_dict['⑤'] = '(5)'
    headnum_dict['⑥'] = '(6)'
    headnum_dict['⑦'] = '(7)'
    headnum_dict['⑧'] = '(8)'
    headnum_dict['⑨'] = '(9)'
    headnum_dict['⑩'] = '(10)'
    headnum_dict['⑪'] = '(11)'
    headnum_dict['⑫'] = '(12)'
    headnum_dict['⑬'] = '(13)'
    headnum_dict['⑭'] = '(14)'
    headnum_dict['⑮'] = '(15)'
    headnum_dict['⑯'] = '(16)'
    headnum_dict['⑰'] = '(17)'
    headnum_dict['⑱'] = '(18)'
    headnum_dict['⑲'] = '(19)'
    headnum_dict['⑳'] = '(20)'
    headnum_dict['㉑'] = '(21)'
    headnum_dict['㉒'] = '(22)'
    headnum_dict['㉓'] = '(23)'
    headnum_dict['㉔'] = '(24)'
    headnum_dict['㉕'] = '(25)'
    headnum_dict['㉖'] = '(26)'
    headnum_dict['㉗'] = '(27)'
    headnum_dict['㉘'] = '(28)'
    headnum_dict['㉙'] = '(29)'
    headnum_dict['㉚'] = '(30)'
    headnum_dict['가.'] = '(a)'
    headnum_dict['나.'] = '(b)'
    headnum_dict['다.'] = '(c)'
    headnum_dict['라.'] = '(d)'
    headnum_dict['마.'] = '(e)'
    headnum_dict['바.'] = '(f)'
    headnum_dict['사.'] = '(g)'
    headnum_dict['아.'] = '(h)'
    headnum_dict['자.'] = '(i)'
    headnum_dict['차.'] = '(j)'
    headnum_dict['카.'] = '(k)'
    headnum_dict['타.'] = '(l)'
    headnum_dict['파.'] = '(m)'
    headnum_dict['하.'] = '(n)'
    headnum_dict['거.'] = '(o)'
    headnum_dict['너.'] = '(p)'
    headnum_dict['더.'] = '(q)'
    headnum_dict['러.'] = '(r)'
    headnum_dict['머.'] = '(s)'
    headnum_dict['버.'] = '(t)'
    headnum_dict['서.'] = '(u)'
    headnum_dict['어.'] = '(v)'
    headnum_dict['저.'] = '(w)'
    headnum_dict['처.'] = '(x)'
    headnum_dict['커.'] = '(y)'
    headnum_dict['터.'] = '(z)'
    headnum_dict['퍼.'] = '(A)'
    headnum_dict['허.'] = '(B)'

   
create_headnum_dic()
    
def check_headnum(org, mean, reg, reg_type):    
    s_match = reg.search(org)    
    if (s_match != None) :
        t_match = reg.search(mean)
        if (t_match == None) :
            if(reg_type == 0):
                mean = f'{s_match.group()} {mean}'
            else:
                if(s_match.group() in headnum_dict):                    
                    mean = f'{headnum_dict[s_match.group()]} {mean}'                    
    return mean
    

In [151]:
def translate(text):
    if text == '' or text == None :
        return '번역할 문장이 없습니다.'    
    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0).to(DEVICE)    
    output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5).to(DEVICE)    
    output = tokenizer.decode(output[0], skip_special_tokens=True)    
    output = after_proc(output)
    output = output[0].upper() + output[1:]
    output = check_headnum(text, output, reg_ho, 0)
    output = check_headnum(text, output, reg_hang, 1)
    output = check_headnum(text, output, reg_mog, 1)
    
    return output

In [ ]:
tok = get_kobart_tokenizer()

In [162]:
#print(translate('⑤ 건널목 개량촉진법 일부를 다음과 같이 개정한다.'))
#print(translate('3. “피4해종교단체”란 10ㆍ27법난으로 인하여 종교적 존엄성과 명예를 훼손당한 대한불교 조계종 및 10ㆍ27법난 당시 피해자가 소속된 사찰을 말한다.'))
#print(translate('③ 위원회는 위원장을 포함한 11인 이내의 위원으로 구성한다.'))
#print(translate('② 제1항에 따라 의료지원금을 반환할 자가 해당 금액을 반환하지 아니한 때에는 국세 체납처분의 예에 따른다.'))
#print(translate('③ 개량건널목의 지정기준에 관하여 필요한 사항은 대통령령으로 정한다.'))
print(translate('필수적 공동소송인 공유물분할청구소송에서, 공동소송인 중 1인에 소송요건의 흠이 있으면 전체 소송이 부적법하게 되는지 여부(적극)'))
#print(translate('11. 고용보험기금의 운용 및 재산 등의 관리'))

In a In a lawsuit for claiming the division of co-owned property, which is an essential joint litigation, whether or not the entire litigation becomes inappropriate if one of the joint litigants has flaws in the litigation requirements (affirmative)
3
In a lawsuit for claiming the division of co-owned property, which is an essential joint litigation, whether or not the entire litigation becomes inappropriate if one of the joint litigants has flaws in the litigation requirements (affirmative)


In [155]:
#text = '법 제61조제4항에서 "대통령령으로 정하는 방법"이란 그 밖의 시설물 및 구축물(토지 또는 건물과 일괄하여 평가하는 것을 제외한다)에 대하여 그것을 다시 건축하거나 다시 취득할 경우에 소요되는 가액(이하 이 항에서 "재취득가액등"이라 한다)에서 그것의 설치일부터 평가기준일까지의 기획재정부령으로 정하는 감가상각비상당액을 뺀 것을 말한다.'
text = '법 제61조제4항에서 "대통령령으로 정하는 방법"이란 그 밖의 시설물 및 구축물(토지 또는 건물과 일괄하여 평가하는 것을 제외한다)에 대하여 그것을 다시 건축하거나 다시 취득할 경우에 소요되는 가액(이하 이 항에서 "재취득가액등"이라 한다)에서 그것의 설치일부터 평가기준일까지의 기획재정부령으로 정하는 감가상각비상당액을 뺀 것을 말한다.'
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0).to(DEVICE)
print(input_ids)
print('input_ids'+'='*20)
output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5).to(DEVICE)
print(output)
print('output-1'+'='*20)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)
print('output-2'+'='*20)

tensor([[14326, 27586,   248, 12178, 12147,   251, 13603, 14030, 18690, 14283,
         10326, 14027, 29445, 14909,   233, 15548, 14028, 27614, 15983, 10675,
         14280, 15733, 10675,   239, 28788, 15056, 16442,  9120, 27101, 14330,
         29269, 14374, 15794, 16728, 15213, 18275, 16960, 14461, 16671, 15570,
         14461, 17769, 13594, 19963, 20877, 14343, 14039, 11724, 19191, 14025,
         14577, 14030, 14050, 12080, 12819,  9990,  8981, 11724, 10001,   233,
         14096, 16501, 16352, 14654, 12024, 15218, 15243, 14653, 16859, 15364,
         12024, 22744, 10326, 14027, 29445, 14164,  8981, 11224,  8982, 11028,
         11224,  9770, 17467,  1700, 11067, 14374, 18520, 14130]],
       device='cuda:0')
input_ids====================
tensor([[    2, 29672, 18090, 14879,   308, 14050,   308,   300, 16160, 20223,
         17065,   313, 15972,   298, 22720,   297, 17752, 19306,   320, 15073,
           313, 15972, 18509, 15265, 15555, 15585, 15529, 28397, 16651,   300,
          

In [37]:
def translate(text):
    if text == '' or text == None :
        return '번역할 문장이 없습니다.'    
    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0).to(DEVICE)    
    output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5).to(DEVICE)    
    output = tokenizer.decode(output[0], skip_special_tokens=True)    
    output = after_proc(output)
    output = check_headnum(text, output, reg_ho, 0)
    output = check_headnum(text, output, reg_hang, 1)
    output = check_headnum(text, output, reg_mog, 1)
    
    return output

In [20]:
tok = get_kobart_tokenizer()

using cached model. /etc/jupyterhub/pythonex/Paper_2023/KoBART/.cache/kobart_base_tokenizer_cased_cf74400bce.zip


In [130]:
#print(translate('⑤ 건널목 개량촉진법 일부를 다음과 같이 개정한다.'))
#print(translate('3. “피4해종교단체”란 10ㆍ27법난으로 인하여 종교적 존엄성과 명예를 훼손당한 대한불교 조계종 및 10ㆍ27법난 당시 피해자가 소속된 사찰을 말한다.'))
#print(translate('③ 위원회는 위원장을 포함한 11인 이내의 위원으로 구성한다.'))
#print(translate('② 제1항에 따라 의료지원금을 반환할 자가 해당 금액을 반환하지 아니한 때에는 국세 체납처분의 예에 따른다.'))
#print(translate('③ 개량건널목의 지정기준에 관하여 필요한 사항은 대통령령으로 정한다.'))
print(translate('필수적 공동소송인 공유물분할청구소송에서, 공동소송인 중 1인에 소송요건의 흠이 있으면 전체 소송이 부적법하게 되는지 여부(적극)'))

In a In a lawsuit for claiming the division of co-owned property, which is an essential joint litigation, whether or not the entire litigation becomes inappropriate if one of the joint litigants has flaws in the litigation requirements (affirmative)
0
In a In a lawsuit for claiming the division of co-owned property, which is an essential joint litigation, whether or not the entire litigation becomes inappropriate if one of the joint litigants has flaws in the litigation requirements (affirmative)


In [9]:
#text = '법 제61조제4항에서 "대통령령으로 정하는 방법"이란 그 밖의 시설물 및 구축물(토지 또는 건물과 일괄하여 평가하는 것을 제외한다)에 대하여 그것을 다시 건축하거나 다시 취득할 경우에 소요되는 가액(이하 이 항에서 "재취득가액등"이라 한다)에서 그것의 설치일부터 평가기준일까지의 기획재정부령으로 정하는 감가상각비상당액을 뺀 것을 말한다.'
text = '「장애인복지법」에 따라 등록된 시각ㆍ청각장애인이 생활하는 가정의 수상기'
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0).to(DEVICE)
print(input_ids)
print('input_ids'+'='*20)
output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5).to(DEVICE)
print(output)
print('output-1'+'='*20)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)
print('output-2'+'='*20)

tensor([[ 1700,  1834, 22130, 15997, 10840,  1835, 11786, 14302, 16270,  9908,
         16378,  1286, 12681,  8982, 17405, 14635, 14923, 14049, 27839, 16135,
          9264]], device='cuda:0')
input_ids====================
tensor([[    2, 16132,  1700, 16132,  1700, 17223,   315,   314, 22692, 22911,
         21582, 17884, 17223,   303, 18192,   299, 20676,   303, 14879,   300,
         22911, 17065, 27621, 14889, 20676,   304, 16160, 22911, 20290, 15868,
           316, 27051,   320,  1700, 19732,   311,   296,   304, 16651,   299,
          1700, 15463,  1700, 19001, 15562, 17163,  1700, 19732,   311,   296,
           304, 16651,   299, 17065, 27621, 14889,  1700, 16651,   302, 15868,
         21569, 17752, 17065, 19512,   314,   316, 15195,   315, 27141, 21235,
         14603, 22648,  1700, 14889, 16996, 16997,   301, 15562,   300, 19524,
         15073, 27621, 14889,   314, 20676,   304, 16160, 15529, 15868, 20775,
         17761,   304, 15555, 15972, 25674, 28597, 15972,     1]],